In [71]:
import numpy as np

def log_si_probs(tags, word_indices, log_forward, log_backward, log_forward_val, transition, emission):
    si_probabilities = np.full((len(tags), len(word_indices)-1, len(tags)), -np.inf)

    for i in range(len(word_indices)-1):
        for j in range(len(tags)):
            for k in range(len(tags)):
                si_probabilities[j, i, k] = (
                    log_forward[j, i] +
                    log_backward[k, i+1] +
                    transition[j, k] +
                    emission[k, word_indices[i+1]] -
                    log_forward_val
                )
    return si_probabilities



def log_si_probs_vec(tags, word_indices, log_forward, log_backward, log_forward_val, transition, emission):
    """
    Vectorized implementation of log_si_probs function.
    
    Parameters:
    - tags: List of possible tags
    - word_indices: Sequence of word indices
    - log_forward: Log forward probabilities
    - log_backward: Log backward probabilities
    - log_forward_val: Log forward value
    - transition: Transition log probabilities matrix
    - emission: Emission log probabilities matrix
    
    Returns:
    - si_probabilities: Vectorized log probabilities tensor
    """
    # Determine dimensions
    num_tags = len(tags)
    num_words = len(word_indices) - 1
    
    # Create the output tensor filled with -inf
    si_probabilities = np.full((num_tags, num_words, num_tags), -np.inf)
    
    next_word_indices = word_indices[1:]
    
    # Compute the log probabilities using broadcasting
    # This replaces the nested loops with vectorized operations
        # Compute the full tensor of log probabilities for this word position
    return (
        log_forward[:, :-1, np.newaxis] +  # first tag dimension
        log_backward[:, 1:].T[np.newaxis, :] +  # second tag dimension
        transition[:, np.newaxis, :] +  # transition probabilities
        emission[:, next_word_indices].T[np.newaxis, :] -  # emission for next word
        log_forward_val
    )
    
    return si_probabilities




In [72]:
import numpy as np

# Define the parameters
tags = ['Noun', 'Verb', 'Adjective']
word_sequence = ['the', 'quick', 'brown', 'fox']
word_indices = [0, 1, 2, 3]  # corresponding indices in a hypothetical vocabulary
log_forward = np.array([
    [1, 2, 3, 4],  # Values for Noun
    [5, 6, 7, 8],  # Values for Verb
    [9, 10, 11, 12]  # Values for Adjective
])
log_backward = np.array([
    [120, 110, 100, 90],  # Values for Noun
    [80, 70, 60, 50],  # Values for Verb
    [40, 30, 20, 10]  # Values for Adjective
])
log_forward_val = 10

# Define transition matrix with distinct values
transition = np.array([
    [1, 2, 3],  # From Noun to Noun, Verb, Adjective
    [4, 5, 6],  # From Verb to Noun, Verb, Adjective
    [7, 8, 9]   # From Adjective to Noun, Verb, Adjective
])
# Define emission matrix with distinct values
vocab_size = max(word_indices) + 1
emission = np.array([
    [11, 22, 33, 44],  # Emissions for Noun
    [88, 77, 66, 55],  # Emissions for Verb
    [99, 111, 222, 333]   # Emissions for Adjective
])

# Print input values
print("Tags:", tags)
print("Word Indices:", word_indices)
print("Word Sequence:", word_sequence)
print("Forward Matrix:\n", log_forward)
print("Backward Matrix:\n", log_backward)
print("Forward Value:", log_forward_val)
print("Transition Matrix:\n", transition)
print("Emission Matrix:\n", emission)

# Use the original function
si_probabilities = log_si_probs(tags, word_indices, log_forward, log_backward, log_forward_val, transition, emission)
print("SI Probabilities from log_si_probs:\n", si_probabilities)

# Use the vectorized function
si_probabilities_vec = log_si_probs_vec(tags, word_indices, log_forward, log_backward, log_forward_val, transition, emission)
print("SI Probabilities from log_si_probs_vec:\n", si_probabilities_vec)

# Check if the outputs are equivalent
print("Are the outputs equivalent?", np.allclose(si_probabilities, si_probabilities_vec))


Tags: ['Noun', 'Verb', 'Adjective']
Word Indices: [0, 1, 2, 3]
Word Sequence: ['the', 'quick', 'brown', 'fox']
Forward Matrix:
 [[ 1  2  3  4]
 [ 5  6  7  8]
 [ 9 10 11 12]]
Backward Matrix:
 [[120 110 100  90]
 [ 80  70  60  50]
 [ 40  30  20  10]]
Forward Value: 10
Transition Matrix:
 [[1 2 3]
 [4 5 6]
 [7 8 9]]
Emission Matrix:
 [[ 11  22  33  44]
 [ 88  77  66  55]
 [ 99 111 222 333]]
SI Probabilities from log_si_probs:
 [[[124. 140. 135.]
  [126. 120. 237.]
  [128. 100. 339.]]

 [[131. 147. 142.]
  [133. 127. 244.]
  [135. 107. 346.]]

 [[138. 154. 149.]
  [140. 134. 251.]
  [142. 114. 353.]]]
SI Probabilities from log_si_probs_vec:
 [[[124 140 135]
  [126 120 237]
  [128 100 339]]

 [[131 147 142]
  [133 127 244]
  [135 107 346]]

 [[138 154 149]
  [140 134 251]
  [142 114 353]]]
Are the outputs equivalent? True


In [60]:
import numpy as np

# Parameters
num_tags = 3
num_words = 16
word_indices = np.array([i for i in range(16)])  # Example word indices
log_forward = np.random.rand(num_tags, num_words)  # Shape: (num_tags, num_words)
log_backward = np.random.rand(num_tags, num_words)  # Shape: (num_tags, num_words)
log_transition = np.random.rand(num_tags, num_tags)  # Shape: (num_tags, num_tags)
log_emission = np.random.rand(num_tags, num_words)  # Shape: (num_tags, num_words)
log_forward_val = np.random.rand()  # Scalar for simplicity

# Prepare for vectorized computation
next_word_indices = word_indices[1:]  # Shape: (num_words - 1,)

# Expand dimensions to align shapes for broadcasting
log_forward_expanded = log_forward[:, :-1, np.newaxis]
log_backward_expanded = log_backward[:, 1:].T[np.newaxis, :]
log_transition_expanded = log_transition[:, np.newaxis, :]
log_emission_expanded = log_emission[:, next_word_indices].T[np.newaxis, :]

# Print results to verify shapes
print("log_forward shape:", log_forward.shape)
print("log_backward shape:", log_backward.shape)
print("log_transition shape:", log_transition.shape)
print("log_emission shape:", log_emission.shape)
print("log_forward_expanded shape:", log_forward_expanded.shape)
print("log_backward_expanded shape:", log_backward_expanded.shape)
print("log_transition_expanded shape:", log_transition_expanded.shape)
print("log_emission_expanded shape:", log_emission_expanded.shape)

# Fully vectorized computation
si_probs = (
    log_forward_expanded +
    log_backward_expanded +
    log_transition_expanded +
    log_emission_expanded -
    log_forward_val
)  # Final shape: (num_tags, num_words-1, num_tags)
print("si_probs shape:", si_probs.shape)


print(si_probs)


log_forward shape: (3, 16)
log_backward shape: (3, 16)
log_transition shape: (3, 3)
log_emission shape: (3, 16)
log_forward_expanded shape: (3, 15, 1)
log_backward_expanded shape: (1, 15, 3)
log_transition_expanded shape: (3, 1, 3)
log_emission_expanded shape: (1, 15, 3)
si_probs shape: (3, 15, 3)
[[[1.51029713 1.38980638 1.52449204]
  [2.29366963 1.08334145 2.11525303]
  [1.56355873 2.0600615  1.57484353]
  [0.89286649 1.14084519 1.61973685]
  [2.14841464 2.60954947 2.17309765]
  [1.60233251 2.08336332 1.60663009]
  [1.7372244  2.71781189 1.16597556]
  [2.12092602 1.81901328 1.62118963]
  [1.74210682 1.38239048 2.04785704]
  [2.08971187 1.98999997 1.93478329]
  [2.31690699 2.4274563  1.91590583]
  [1.45032894 2.48323461 2.02261856]
  [0.6601869  2.02478702 1.68270745]
  [1.74223453 2.78878149 1.70630344]
  [2.69219542 2.88730913 2.36281998]]

 [[2.03880481 1.89595267 2.57276611]
  [2.02446347 0.79177391 2.36581327]
  [1.29469551 1.76883689 1.82574669]
  [1.27226576 1.49788306 2.518902